In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("carsclean.csv")
df.head()

,city,car_maker,model,year,condition,kilometers,transmission,fuel,color,pay_method,price
0,Al Riyadh,Toyota,Rush,2019,Used,14999.5,Automatic,Gasoline,White,Cash or Installments,58375.0
1,Al Riyadh,Kia,Carnival,2006,Used,0.0,Automatic,Gasoline,Grey,Cash Only,10000.0
2,Al Riyadh,Toyota,Camry,2014,Used,24999.5,Automatic,Gasoline,White,Installments Only,58375.0
3,Al Riyadh,Ford,Fusion,2010,Used,200000.0,Automatic,Gasoline,White,Cash Only,22425.4
4,Al Riyadh,Ford,Taurus,2018,Used,5000.0,Automatic,Gasoline,Grey,Cash Only,22425.4


In [3]:
df.info()   #Hiç bir sütunda null değeri olmadığını görüyoruz.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          560 non-null    object 
 1   car_maker     560 non-null    object 
 2   model         560 non-null    object 
 3   year          560 non-null    int64  
 4   condition     560 non-null    object 
 5   kilometers    560 non-null    float64
 6   transmission  560 non-null    object 
 7   fuel          560 non-null    object 
 8   color         560 non-null    object 
 9   pay_method    560 non-null    object 
 10  price         560 non-null    float64
dtypes: float64(2), int64(1), object(8)
memory usage: 48.2+ KB


In [4]:
columns_list = df.columns  #Sütun isimlerini columns_list değişkeninde depoladık.
                           #Listenin her bir ögesinde dolaşarak sütun isimlerini alıp
                           #O sütunda kaç çeşit değişken olduğunu yazdırdık
for column in columns_list:
  print(column)
  print(df[column].value_counts())
  print("--------------------")

city
Al Riyadh    560
Name: city, dtype: int64
--------------------
car_maker
Toyota      140
Ford        120
BMW          80
Kia          60
Renault      40
Dodge        20
GMC          20
Cadillac     20
Suzuki       20
Mazda        20
Nissan       20
Name: car_maker, dtype: int64
--------------------
model
Fusion            40
740               40
Camry             40
Expedition        20
Dokker Van        20
3                 20
Opirus            20
730               20
Escalade          20
Rio               20
Prado             20
X5                20
Fortuner          20
Envoy             20
Taurus            20
Durango           20
Symbol            20
Yaris             20
Other             20
Carnival          20
Rush              20
Patrol            20
Explorer          20
Avalon            20
Crown Victoria    20
Name: model, dtype: int64
--------------------
year
2008    100
2014     80
2016     60
2013     60
2007     60
2019     40
2009     40
2018     20
2015     20
2012